In [14]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            images.append(img)
            # Assign label based on filename
            if 'cat' in filename:
                labels.append(0)
            elif 'dog' in filename:
                labels.append(1)
        else:
            print(f"Failed to read image: {img_path}")
    print(f"Loaded {len(images)} images from {folder}")
    return images, labels

def load_dataset(data_dir):
    images, labels = load_images_from_folder(data_dir)
    if len(images) == 0:
        print(f"No images found in {data_dir}. Please check the path.")
    return np.array(images), np.array(labels)

# Load train and test datasets
train_images, train_labels = load_dataset('dogs-vs-cats/train')
test_images, test_labels = load_dataset('dogs-vs-cats/test1')

# Check if any images were loaded
print(f"Train images shape: {train_images.shape}")
print(f"Test images shape: {test_images.shape}")

if train_images.size == 0 or test_images.size == 0:
    raise ValueError("No images loaded. Please check the dataset path and file permissions.")


Loaded 25000 images from dogs-vs-cats/train
Loaded 12500 images from dogs-vs-cats/test1
Train images shape: (25000, 64, 64)
Test images shape: (12500, 64, 64)
Train images flattened shape: (25000, 4096)
Test images flattened shape: (12500, 4096)


In [ ]:
# Normalize the images
train_images = train_images / 255.0
test_images = test_images / 255.0

# Flatten the images for the SVM
train_images_flattened = train_images.reshape(len(train_images), -1)
test_images_flattened = test_images.reshape(len(test_images), -1)

print(f"Train images flattened shape: {train_images_flattened.shape}")
print(f"Test images flattened shape: {test_images_flattened.shape}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images_flattened, train_labels, test_size=0.2, random_state=42)

# Train the SVM model
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Validate the model
y_val_pred = svm_model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("Classification Report:\n", classification_report(y_val, y_val_pred))


In [ ]:
# Evaluate the model on the test set
y_test_pred = svm_model.predict(test_images_flattened)
print("Test Accuracy:", accuracy_score(test_labels, y_test_pred))
print("Classification Report:\n", classification_report(test_labels, y_test_pred))

import matplotlib.pyplot as plt

def plot_sample_images(images, labels, predictions=None):
    plt.figure(figsize=(10, 10))
    for i in range(25):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        if predictions is None:
            plt.xlabel("Actual: " + str(labels[i]))
        else:
            plt.xlabel(f"Actual: {labels[i]} Pred: {predictions[i]}")
    plt.show()

# Plot some test images along with their predictions
plot_sample_images(test_images, test_labels, y_test_pred)
